### Aufgabe 8

Erstellen Sie ein neues Notebook, `Kontrollierte_Wohnungsbeluefung.ipynb`, in dem Sie das folgende Problem behandeln:

Das Erdgeschoss der betrachteten Wohnung hat eine Küche ($12\,m^2$), einen Flur ($6.5\,m^2$), ein Wohnzimmer ($26.8\,m^2$), ein WC ($3.5\,m^2$) <br> und einen Abstellraum ($4.2\,m^2$).

Im Obergeschoss der betrachteten Wohnung gibt es ein Bad ($11.5\,m^2$), ein Elternschlafzimmer ($13.25\,m^2$), und zwei Kinderzimmer ($12.5\,m^2$ und $11.25\,m^2$). Der Flur hat eine Fläche von $4.5\,m^2$.

1. Erstellen Sie einen DataFrame für die gesamte Wohnung (z.B. `df`),<br>
   und Kennzeichnen Sie Räume aus EG und OG sowie die Zu- und Ablufträume
3. Berechnen Sie die Nutzfläche der Wohnung und den erforderlichen Zuluftvolumenstrom `dV_zu`.<br>
   Die benötigte Formel lautet:
   
   $$
     \dot{V}_\text{zu} = -0.001\,A_\text{N}^2 + 1.15\,A_\text{N} + 20
   $$
   
   Mit $A_\text{N}$: Nutzfläche der Wohneinheit in $m^2$, $\dot{V}_\text{zu}$: Volumenstrom in $\frac{m^3}{h}$

3. Erzeugen Sie einen DataFrame für die Ablufträume und ermitteln Sie den erforderlichen Abluftvolumenstrom.
4. Geben sie den erforderlichen Gesamtvolumenstrom an und verteilen Sie Zu- und Abluft auf die Räume.


### Lösung

In [1]:
# Ihre Lösung beginnt hier.
import pandas as pd
import numpy as np

1. DataFrame für die Wohnung mit allen bekannten Daten:

In [2]:
df = pd.DataFrame(
    {
        'Geschoss': ['EG','EG','EG','EG','EG','OG','OG','OG','OG','OG'],
        'Raum': [
            'Küche', 'Flur_eg', 'Wohnzimmer', 'WC', 'Abstellraum',
            'Bad', 'Eltern', 'Kind_1', 'Kind_2', 'Flur_og'
        ],
        'Fläche': [12.0, 6.5, 26.8, 3.5, 4.2,11.5, 13.25, 12.5, 11.25, 4.5],
        'Art': ['Ab','-','Zu','Ab','Ab','Ab','Zu','Zu','Zu','-']
    }
)

df

,Geschoss,Raum,Fläche,Art
0,EG,Küche,12.00,Ab
1,EG,Flur_eg,6.50,-
2,EG,Wohnzimmer,26.80,Zu
3,EG,WC,3.50,Ab
4,EG,Abstellraum,4.20,Ab
5,OG,Bad,11.50,Ab
6,OG,Eltern,13.25,Zu
7,OG,Kind_1,12.50,Zu
8,OG,Kind_2,11.25,Zu
9,OG,Flur_og,4.50,-


2. Nutzfläche und Zuluftvolumentrom berechnen

In [4]:
A_N = df.Fläche.sum()
A_N

106.0

In [5]:
dV_zu = -0.001*A_N**2 + 1.15*A_N + 20
dV_zu

130.664

3. Dataframe für die Ablufträume und Abluftvolumenstrom aus `df` erzeugen und ergänzen, Abluftvolumenstrom berechnen:

In [6]:
df_ab = df[df.Art == 'Ab'].copy()
df_ab['dV_ab'] = [50,25,25,50]
df_ab

,Geschoss,Raum,Fläche,Art,dV_ab
0,EG,Küche,12.0,Ab,50
3,EG,WC,3.5,Ab,25
4,EG,Abstellraum,4.2,Ab,25
5,OG,Bad,11.5,Ab,50


In [7]:
dV_ab = df_ab.dV_ab.sum()
dV_ab

150

4. Gesamtvolumenstrom ermitteln und auf die Räume verteilen:

In [8]:
dV_ges = max(dV_zu,dV_ab)
dV_ges

150

Erzeugen eines DataFrame für die Zulufträume und Ergänzung um die Faktoren, die Volumenströme und die Luftwechselzahlen für jeden Zuluftraum:

In [9]:
df_zu = df[df.Art == 'Zu'].copy()
df_zu['Faktor'] = [3,2,2,2]
df_zu['dV_zu'] = df_zu.Faktor/df_zu.Faktor.sum()*dV_ges
df_zu['Lw'] = df_zu.dV_zu/(df_zu.Fläche*2.5)

df_zu

,Geschoss,Raum,Fläche,Art,Faktor,dV_zu,Lw
2,EG,Wohnzimmer,26.80,Zu,3,50.000000,0.746269
6,OG,Eltern,13.25,Zu,2,33.333333,1.006289
7,OG,Kind_1,12.50,Zu,2,33.333333,1.066667
8,OG,Kind_2,11.25,Zu,2,33.333333,1.185185


Neuberechnung der Volumenströme für die Ablufträume

Das ist in diesem Fall eigentlich nicht nötig!

In [10]:
df_ab.dV_ab = df_ab.dV_ab/dV_ab*dV_ges
df_ab['Lw'] = df_ab.dV_ab/(df_ab.Fläche*2.5)

df_ab

,Geschoss,Raum,Fläche,Art,dV_ab,Lw
0,EG,Küche,12.0,Ab,50.0,1.666667
3,EG,WC,3.5,Ab,25.0,2.857143
4,EG,Abstellraum,4.2,Ab,25.0,2.380952
5,OG,Bad,11.5,Ab,50.0,1.739130


### Zusammenführen der Teile:

Sind die beiden DataFrames `df_ab` und `df_zu` berechnet, so ist man mit der Arbeit eigentlich fertig.

Es ist möglich, die beiden Teile wieder zu einem ganzen zusammenzufügen. Dazu kann man entweder die Funktion `pd.concat([df1,df2,...])` verwenden, oder die Methode `df1.append(df2)`. 

In [11]:
df_ges = pd.concat(
    [df_zu,df_ab],
    sort=False,
)

df_ges.sort_index()

,Geschoss,Raum,Fläche,Art,Faktor,dV_zu,Lw,dV_ab
0,EG,Küche,12.00,Ab,NaN,NaN,1.666667,50.0
2,EG,Wohnzimmer,26.80,Zu,3.0,50.000000,0.746269,NaN
3,EG,WC,3.50,Ab,NaN,NaN,2.857143,25.0
4,EG,Abstellraum,4.20,Ab,NaN,NaN,2.380952,25.0
5,OG,Bad,11.50,Ab,NaN,NaN,1.739130,50.0
6,OG,Eltern,13.25,Zu,2.0,33.333333,1.006289,NaN
7,OG,Kind_1,12.50,Zu,2.0,33.333333,1.066667,NaN
8,OG,Kind_2,11.25,Zu,2.0,33.333333,1.185185,NaN


Man erkennt, dass die Überströmräume hier nicht mehr enthalten sind. Es ist sinnvoll, diese auch noch aufzuführen. Dann schreibt man zum Beispiel:

In [15]:
df_ges = pd.concat(
    [
        df_zu,
        df_ab,
        df[df.Art == '-'], # nur Überströmräume
    ],
    sort=False,
)

df_ges.sort_index()

,Geschoss,Raum,Fläche,Art,Faktor,dV_zu,Lw,dV_ab
0,EG,Küche,12.00,Ab,NaN,NaN,1.666667,50.0
1,EG,Flur_eg,6.50,-,NaN,NaN,NaN,NaN
2,EG,Wohnzimmer,26.80,Zu,3.0,50.000000,0.746269,NaN
3,EG,WC,3.50,Ab,NaN,NaN,2.857143,25.0
4,EG,Abstellraum,4.20,Ab,NaN,NaN,2.380952,25.0
5,OG,Bad,11.50,Ab,NaN,NaN,1.739130,50.0
6,OG,Eltern,13.25,Zu,2.0,33.333333,1.006289,NaN
7,OG,Kind_1,12.50,Zu,2.0,33.333333,1.066667,NaN
8,OG,Kind_2,11.25,Zu,2.0,33.333333,1.185185,NaN
9,OG,Flur_og,4.50,-,NaN,NaN,NaN,NaN


Der DataFrame `df_ges` hat **keinen** sortierten Index!

In [13]:
df_ges

,Geschoss,Raum,Fläche,Art,Faktor,dV_zu,Lw,dV_ab
2,EG,Wohnzimmer,26.80,Zu,3.0,50.000000,0.746269,NaN
6,OG,Eltern,13.25,Zu,2.0,33.333333,1.006289,NaN
7,OG,Kind_1,12.50,Zu,2.0,33.333333,1.066667,NaN
8,OG,Kind_2,11.25,Zu,2.0,33.333333,1.185185,NaN
0,EG,Küche,12.00,Ab,NaN,NaN,1.666667,50.0
3,EG,WC,3.50,Ab,NaN,NaN,2.857143,25.0
4,EG,Abstellraum,4.20,Ab,NaN,NaN,2.380952,25.0
5,OG,Bad,11.50,Ab,NaN,NaN,1.739130,50.0
1,EG,Flur_eg,6.50,-,NaN,NaN,NaN,NaN
9,OG,Flur_og,4.50,-,NaN,NaN,NaN,NaN


Möchte man den Index von `df_ges` sortieren, so muss `.sort_index()` mit dem Parameter `.sort_index(inplace=True)` aufgerufen werden.

In [16]:
df_ges.sort_index(inplace=True)
df_ges

,Geschoss,Raum,Fläche,Art,Faktor,dV_zu,Lw,dV_ab
0,EG,Küche,12.00,Ab,NaN,NaN,1.666667,50.0
1,EG,Flur_eg,6.50,-,NaN,NaN,NaN,NaN
2,EG,Wohnzimmer,26.80,Zu,3.0,50.000000,0.746269,NaN
3,EG,WC,3.50,Ab,NaN,NaN,2.857143,25.0
4,EG,Abstellraum,4.20,Ab,NaN,NaN,2.380952,25.0
5,OG,Bad,11.50,Ab,NaN,NaN,1.739130,50.0
6,OG,Eltern,13.25,Zu,2.0,33.333333,1.006289,NaN
7,OG,Kind_1,12.50,Zu,2.0,33.333333,1.066667,NaN
8,OG,Kind_2,11.25,Zu,2.0,33.333333,1.185185,NaN
9,OG,Flur_og,4.50,-,NaN,NaN,NaN,NaN
